# Connections

In [116]:
import pyodbc
import pandas as pd
from pptx import Presentation
from pptx.util import Cm, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.oxml.xmlchemy import OxmlElement
import win32com.client as win32
import os
import warnings
warnings.filterwarnings("ignore")

TITLE_COLOR = RGBColor(50, 50, 255)
HEADER_BG = RGBColor(42, 42, 42)
HEADER_TEXT = RGBColor(255, 255, 255)
ROW_ALT_BG = RGBColor(247, 247, 247)
ROW_BG = RGBColor(255, 255, 255)

import warnings
warnings.filterwarnings("ignore")

import win32com.client as win32
import os

print(pyodbc.drivers())

server = 'neodocs-sql-server.database.windows.net'
database = 'neodocs-sql-db'
username = 'ndDashboard'
password = 'NeoDocs@2025'

try:
    conn = pyodbc.connect(
        f"DRIVER={{SQL Server}};"
        f"SERVER={server};DATABASE={database};UID={username};PWD={password}"
    )
    print("✅ Connected!")
except Exception as e:
    print("❌ Connection failed:", e)
    raise


['SQL Server', 'Oracle in OraDB21Home1', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)', 'ODBC Driver 18 for SQL Server']
✅ Connected!


# Raw data loader

In [117]:
def load_raw_data(conn, start_date=None, end_date=None, pharma_filter="ALL", load_all=False):

    query = """
    SELECT
        ut.drName,
        ut.orgCity,
        ut.orgState as State,
        ut.specialization,
        ut.age,
        ut.category,
        ut.gender,
        ut.testId,
        ut.oId,
        ut.campDate,
        ut.userValueFlag,
        ut.isDeleted
    FROM dbo.user_tests ut
    LEFT JOIN dbo.aId a ON ut.aId = a.aId
    WHERE ut.isDeleted = 0
      AND ut.category IN ('A1', 'A2', 'A3')

    """

    params = []

    if not load_all:
        query += " AND ut.dateTime >= ? AND ut.dateTime < DATEADD(DAY, 1, ?) "
        params.extend([start_date, end_date])

    if pharma_filter != "ALL":
        query += " AND a.name LIKE '%' + ? + '%' "
        params.append(pharma_filter)

    return pd.read_sql(query, conn, params=params)


# Transformations and Table Builders

In [118]:
def assign_age_group(age):
    if pd.isna(age): return None
    try: age = int(age)
    except: return None

    if age < 30: return "<30"
    if age < 40: return "30-40"
    if age < 50: return "40-50"
    if age < 60: return "50-60"
    if age < 70: return "60-70"
    if age < 80: return "70-80"
    if age < 90: return "80-90"
    return ">90"


def build_group_table(df, group_col, threshold=None):
    if df.empty:
        return pd.DataFrame()

    df = df.copy()

    # AGE GROUP BINNING
    if group_col == "age_group":
        df["age_group"] = df["age"].apply(assign_age_group)

    # GENDER MERGING → only Male / Female
    if group_col == "gender" and "gender" in df.columns:
        df["gender"] = df["gender"].replace({"Other": "Male"})
        df["gender"] = df["gender"].apply(lambda x: "Female" if str(x).lower()=="female" else "Male")

    # SPECIALIZATION THRESHOLD GROUPING
    if group_col == "specialization" and threshold is not None:
        counts = df.groupby("specialization")["testId"].nunique()
        low = counts[counts < threshold].index
        df["specialization"] = df["specialization"].replace({spec: "Others" for spec in low})

    df = df[df[group_col].notna()]

    total_tests = df.groupby(group_col)["testId"].nunique().rename("Total Tests")

    df["camp_key"] = df["oId"].astype(str) + "|" + df["campDate"].astype(str)
    total_camps = df.groupby(group_col)["camp_key"].nunique().rename("Total Camps")

    cat_pivot = (
        df.pivot_table(
            index=group_col,
            columns="category",
            values="testId",
            aggfunc="nunique",
            fill_value=0
        ).reset_index()
    )

    abn = (
        df.groupby(group_col)
        .apply(lambda x: round(100 * (x["userValueFlag"] != 0).sum() /
                               max(x["testId"].nunique(), 1), 2))
        .rename("Abnormality %")
    )

    final = pd.DataFrame({group_col: total_tests.index})
    final = final.merge(total_tests, on=group_col)
    final = final.merge(total_camps, on=group_col)
    final = final.merge(cat_pivot, on=group_col)
    final = final.merge(abn, on=group_col)
    final = final.fillna(0)

    # SORTING & LIMITS
    if group_col == "age_group":
        age_order = ["<30","30-40","40-50","50-60","60-70","70-80","80-90",">90"]
        final[group_col] = pd.Categorical(final[group_col], categories=age_order, ordered=True)
        final = final.sort_values(group_col)

    elif group_col == "State":
        final = final.sort_values("Total Tests", ascending=False).head(13)

    else:
        final = final.sort_values("Total Tests", ascending=False).head(15)

    return final.reset_index(drop=True)


In [119]:
def compute_overview_metrics(raw_df):
    """Compute summary metrics for the overview page."""

    # 1. Total Tests
    total_tests = len(raw_df)

    # 2. Total Doctors (unique combination of drName, state, city, specialization)
    doctor_cols = ['drName', 'State', 'orgCity', 'specialization']
    if all(col in raw_df.columns for col in doctor_cols):
        total_doctors = raw_df[doctor_cols].drop_duplicates().shape[0]
    else:
        total_doctors = 0

    # 3. Total Camps (unique combination of campDate + oId)
    camp_cols = ['campDate', 'oId']
    if all(col in raw_df.columns for col in camp_cols):
        total_camps = raw_df[camp_cols].drop_duplicates().shape[0]
    else:
        total_camps = 0

    # 4. A2+A3 Abnormality % (from a23_abnormal)
    if 'userValueFlag' in raw_df.columns:
        abnormal_count = (raw_df['userValueFlag'] != 0).sum()
        abnormal_pct = round((abnormal_count / total_tests) * 100, 0)
    else:
        abnormal_pct = 0

    # 5. States Covered
    states_covered = raw_df['State'].nunique() if 'State' in raw_df.columns else 0

    return [
        ("Total Tests", f"{total_tests:,}"),
        ("Total Doctors", f"{total_doctors:,}"),
        ("Total Camps", f"{total_camps:,}"),
        ("A2+A3 Abnormality", f"{abnormal_pct}%"),
        ("States Covered", f"{states_covered:,}")
    ]


# Themes, helpers, footers

In [120]:
from pptx import Presentation
from pptx.util import Inches, Pt, Cm
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_AUTO_SHAPE_TYPE
from datetime import datetime

# ---------- THEME COLORS ----------
TEAL_PRIMARY     = RGBColor(16, 168, 168)      # main accent
TEAL_LIGHT       = RGBColor(228, 248, 248)     # light background tint
TEXT_DARK        = RGBColor(32, 32, 32)
TEXT_MUTED       = RGBColor(120, 120, 120)
ROW_LIGHT_1      = RGBColor(246, 250, 250)
ROW_LIGHT_2      = RGBColor(236, 244, 244)
HEADER_BG        = TEAL_PRIMARY
HEADER_TEXT      = RGBColor(255, 255, 255)

FOOTER_STRING    = "Neodocs YCS21 \u2022 Confidential \u2022 Pharma Insights Report"


# ---------- FOOTER ----------
def _add_footer(prs, slide, slide_number: int):
    """
    Adds startup-style footer:
    Left:  brand text
    Right: slide number
    """
    slide_width = prs.slide_width
    slide_height = prs.slide_height
    margin = Inches(0.4)
    footer_top = slide_height - Inches(0.35)

    # Left footer (brand text)
    left_box = slide.shapes.add_textbox(margin, footer_top, slide_width / 2, Inches(0.3))
    tf_left = left_box.text_frame
    p_left = tf_left.paragraphs[0]
    p_left.text = FOOTER_STRING
    p_left.font.size = Pt(9)
    p_left.font.color.rgb = TEXT_MUTED
    p_left.alignment = PP_ALIGN.LEFT

    # Right footer (slide number)
    right_box = slide.shapes.add_textbox(slide_width / 2, footer_top,
                                         slide_width / 2 - margin, Inches(0.3))
    tf_right = right_box.text_frame
    p_right = tf_right.paragraphs[0]
    p_right.text = str(slide_number)
    p_right.font.size = Pt(9)
    p_right.font.color.rgb = TEXT_MUTED
    p_right.alignment = PP_ALIGN.RIGHT


# ---------- KPI / OVERVIEW HELPERS ----------
def _extract_kpi_value(overview_data, label, default="N/A"):
    """
    overview_data is expected to be a list of (label, value) tuples.
    We try to match the label case-insensitively.
    """
    label_norm = label.strip().lower()
    for k, v in overview_data:
        if str(k).strip().lower() == label_norm:
            return str(v)
    return str(default)


# ---------- ROW GAUGE HELPERS ----------
def _add_small_row_gauge(slide, pct, left, top, width, height):
    pct = max(0, min(100, float(pct)))

    # Background bar
    bar = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.ROUNDED_RECTANGLE,
        left, top, width, height
    )
    bar.fill.solid()
    bar.fill.fore_color.rgb = RGBColor(234, 243, 243)
    bar.line.fill.background()

    # Filled portion (teal)
    fill_width = width * (pct / 100.0)
    fill = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.ROUNDED_RECTANGLE,
        left, top, fill_width, height
    )
    fill.fill.solid()
    fill.fill.fore_color.rgb = TEAL_PRIMARY
    fill.line.fill.background()

def snake_to_title(s):
    return str(s).replace("_", " ").title()


# Overview slide

In [121]:
# ---------- OVERVIEW SLIDE ----------
def add_overview_slide(prs, overview_data, title="Overview"):
    """
    Startup-modern overview slide:
    - Big centered title with teal accent bar
    - 5 KPI tiles with line-icon style
    - Footer + slide number
    """
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    slide_number = len(prs.slides)

    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # ----- TITLE -----
    title_box = slide.shapes.add_textbox(
        Inches(0.5), Inches(0.4),
        slide_width - Inches(1), Inches(0.8)
    )
    tf = title_box.text_frame
    p = tf.paragraphs[0]
    p.text = title
    p.font.size = Pt(38)
    p.font.bold = True
    p.font.color.rgb = TEXT_DARK
    p.alignment = PP_ALIGN.CENTER

    # Accent bar under title
    accent = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.RECTANGLE,
        Inches(3.0), Inches(1.1),
        slide_width - Inches(6.0), Inches(0.08)
    )
    accent.fill.solid()
    accent.fill.fore_color.rgb = TEAL_PRIMARY
    accent.line.fill.background()

    # Subtitle
    subtitle_box = slide.shapes.add_textbox(
        Inches(0.5), Inches(1.2),
        slide_width - Inches(1), Inches(0.5)
    )
    tf_sub = subtitle_box.text_frame
    p_sub = tf_sub.paragraphs[0]
    p_sub.text = "High-level summary of testing performance"
    p_sub.font.size = Pt(16)
    p_sub.font.color.rgb = TEXT_MUTED
    p_sub.alignment = PP_ALIGN.CENTER

    # ----- KPI TILES -----
    # Use labels that exist in your overview_data
    kpi_labels = [
        "Total Tests",
        "Total Camps",
        "Total Doctors",
        "A2+A3 Abnormality",    # or "Abnormality %" depending on your overview_data
        "States Covered",       # or "Unique States"
    ]

    kpi_items = [
        (label, _extract_kpi_value(overview_data, label, default="N/A"))
        for label in kpi_labels
    ]

    tile_width = Inches(3.0)
    tile_height = Inches(1.55)
    tile_gap = Inches(0.5)

    # ROW 1 (3 centered)
    row1_top = Inches(2.15)
    row1_total = tile_width * 3 + tile_gap * 2
    row1_left = (slide_width - row1_total) / 2

    for i in range(3):
        left = row1_left + i * (tile_width + tile_gap)
        _add_kpi_tile(slide, kpi_items[i][0], kpi_items[i][1],
                      left, row1_top, tile_width, tile_height)

    # ROW 2 (2 centered)
    row2_top = row1_top + tile_height + Inches(0.55)
    row2_total = tile_width * 2 + tile_gap
    row2_left = (slide_width - row2_total) / 2

    for i in range(3, 5):
        left = row2_left + (i - 3) * (tile_width + tile_gap)
        _add_kpi_tile(slide, kpi_items[i][0], kpi_items[i][1],
                      left, row2_top, tile_width, tile_height)

    # ----- FOOTER -----
    _add_footer(prs, slide, slide_number)

    return slide


def _add_kpi_tile(slide, label, value, left, top, width, height):
    """
    Adds a single KPI tile in a startup-modern style:
    - rounded rectangle
    - line-style "icon"
    - label + big value
    """
    # Tile background
    tile = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.ROUNDED_RECTANGLE,
        left, top, width, height
    )
    tile.fill.solid()
    tile.fill.fore_color.rgb = TEAL_LIGHT
    tile.line.color.rgb = RGBColor(235, 240, 240)
    tile.line.width = Pt(1.0)

    # "Line icon" circle on left
    icon_size = Inches(0.35)
    icon_left = left + Inches(0.35)
    icon_top = top + Inches(0.3)

    icon = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.OVAL,
        icon_left, icon_top, icon_size, icon_size
    )
    icon.fill.background()  # no fill
    icon.line.color.rgb = TEAL_PRIMARY
    icon.line.width = Pt(2)

    # Label text (top right)
    label_box = slide.shapes.add_textbox(
        icon_left + icon_size + Inches(0.15),
        top + Inches(0.25),
        width - (icon_size + Inches(0.7)),
        Inches(0.5)
    )
    tf_label = label_box.text_frame
    p_label = tf_label.paragraphs[0]
    p_label.text = label
    p_label.font.size = Pt(14)
    p_label.font.color.rgb = TEXT_MUTED
    p_label.alignment = PP_ALIGN.LEFT

    # Value text (big)
    value_box = slide.shapes.add_textbox(
        icon_left + Inches(0.05),
        top + Inches(0.75),
        width - Inches(0.5),
        Inches(0.7)
    )
    tf_value = value_box.text_frame
    p_value = tf_value.paragraphs[0]
    p_value.text = str(value)
    p_value.font.size = Pt(26)
    p_value.font.bold = True
    p_value.font.color.rgb = TEXT_DARK
    p_value.alignment = PP_ALIGN.LEFT

# df to slides

In [122]:
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.enum.shapes import MSO_AUTO_SHAPE_TYPE
from pptx.util import Inches, Pt

def df_to_slide(prs, df, title_text):
    """
    Startup-modern table slide:
    - Wider table & wider key columns
    - Abnormality % gauge INSIDE the last column (side-by-side with value)
    - Gauges for first 15 rows, correctly centered even with many rows
    - Table card auto resizes based on actual table size
    - All text vertically centered
    - No badges, no legend
    - Footer + slide number
    """

    if df is None or df.empty:
        return

    rows, cols = df.shape

    slide = prs.slides.add_slide(prs.slide_layouts[6])
    slide_number = len(prs.slides)

    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # ---------- TITLE ----------
    title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.35),
                                         slide_width - Inches(1), Inches(0.7))
    tf = title_box.text_frame

    p = tf.paragraphs[0]
    p.text = snake_to_title(title_text)
    p.alignment = PP_ALIGN.CENTER
    p.font.size = Pt(32)
    p.font.bold = True
    p.font.color.rgb = TEXT_DARK

    accent = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.RECTANGLE,
        Inches(2.0), Inches(1.0),
        slide_width - Inches(5.0), Inches(0.07)
    )
    accent.fill.solid()
    accent.fill.fore_color.rgb = TEAL_PRIMARY
    accent.line.fill.background()

    # ---------- TABLE LAYOUT ----------
    table_left = Inches(0.1)
    table_top = Inches(1.5)
    table_width = slide_width - Inches(0.4)
    table_height = slide_height - Inches(2.4)

    # Card background (to be resized later)
    card = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.ROUNDED_RECTANGLE,
        table_left - Inches(0.3),
        table_top - Inches(0.3),
        table_width + Inches(0.3),
        table_height + Inches(0.3)
    )
    card.fill.solid()
    card.fill.fore_color.rgb = RGBColor(252, 253, 253)
    card.line.color.rgb = RGBColor(235, 240, 240)
    card.line.width = Pt(1.0)

    # Create table
    table_shape = slide.shapes.add_table(
        rows + 1,
        cols,
        table_left,
        table_top,
        table_width,
        table_height
    )
    table = table_shape.table

    # ---------- AUTO FONT SIZE ----------
    if cols <= 3:
        font_size = 24
    elif cols == 4:
        font_size = 22
    elif cols == 5:
        font_size = 20
    elif cols == 6:
        font_size = 16
    else:
        font_size = 14

    header_font_size = font_size + 2

    # ---------- COLUMN WIDTHS ----------
    first_pct = 0.24
    last_pct  = 0.14

    total_width = table_width

    first_col_width = int(total_width * first_pct)
    last_col_width  = int(total_width * last_pct)

    remaining_width = total_width - first_col_width - last_col_width
    middle_cols = cols - 2

    if middle_cols > 0:
        other_col_width = remaining_width // middle_cols
    else:
        other_col_width = remaining_width

    table.columns[0].width = first_col_width
    if cols > 1:
        table.columns[cols - 1].width = last_col_width
    for c in range(1, cols - 1):
        table.columns[c].width = other_col_width

    # widen key columns
    for name in ["Total Tests", "Total Camps"]:
        if name in df.columns:
            table.columns[df.columns.get_loc(name)].width = Inches(1.35)

    # widen Abnormality column for gauge
    if "Abnormality %" in df.columns:
        table.columns[df.columns.get_loc("Abnormality %")].width = Inches(1.85)

    # ---------- HEADER ----------
    for c, col_name in enumerate(df.columns):
        cell = table.cell(0, c)
        cell.text = snake_to_title(col_name)
        cell.fill.solid()
        cell.fill.fore_color.rgb = HEADER_BG

        tf_header = cell.text_frame
        tf_header.word_wrap = False

        p_header = tf_header.paragraphs[0]
        p_header.font.bold = True
        p_header.font.size = Pt(header_font_size)
        p_header.font.color.rgb = HEADER_TEXT
        p_header.alignment = PP_ALIGN.CENTER

        cell.vertical_anchor = MSO_ANCHOR.MIDDLE

    # ---------- DATA ROWS ----------
    def _is_numeric(val):
        try:
            float(str(val).replace('%', '').replace(',', '').strip())
            return True
        except:
            return False

    for r in range(rows):
        for c in range(cols):
            cell = table.cell(r + 1, c)
            value = df.iat[r, c]
            if c == 0 and not _is_numeric(value):
                cell.text = snake_to_title(value)
            else:
                cell.text = str(value)

            tf_cell = cell.text_frame
            tf_cell.word_wrap = False

            p_cell = tf_cell.paragraphs[0]
            p_cell.font.size = Pt(font_size)
            p_cell.font.color.rgb = TEXT_DARK

            # left align Abnormality column to make room for gauge
            if df.columns[c] == "Abnormality %":
                p_cell.alignment = PP_ALIGN.LEFT
            else:
                p_cell.alignment = PP_ALIGN.RIGHT if _is_numeric(value) else PP_ALIGN.LEFT

            cell.vertical_anchor = MSO_ANCHOR.MIDDLE

            cell.fill.solid()
            cell.fill.fore_color.rgb = ROW_LIGHT_1 if r % 2 == 0 else ROW_LIGHT_2

    # ---------- FINAL ROW HEIGHT RE-CALC (forces real layout) ----------
    prs.slide_width  # layout trigger
    row_heights = [table.rows[i].height for i in range(rows + 1)]
    total_table_height = sum(row_heights)

    # ---------- INLINE GAUGES WITH TRUE CENTERING ----------
    if "Abnormality %" in df.columns:
        ab_col_index = df.columns.get_loc("Abnormality %")

        def _extract_numeric_pct(val):
            try:
                s = str(val).replace('%', '').replace(',', '').strip()
                return float(s)
            except:
                return None

        max_gauges = 15
        gauge_width = Inches(0.70)
        gauge_height = Inches(0.12)

        ab_col_left = table_left + sum(table.columns[i].width for i in range(ab_col_index))

        cumulative_height = row_heights[0]

        for r in range(min(rows, max_gauges)):
            raw_val = df.iat[r, ab_col_index]
            pct_val = _extract_numeric_pct(raw_val)
            if pct_val is None:
                cumulative_height += row_heights[r+1]
                continue

            row_height = row_heights[r+1]
            row_center = table_top + cumulative_height + (row_height / 2)

            gauge_top = row_center - (gauge_height / 2)
            gauge_left = ab_col_left + table.columns[ab_col_index].width - gauge_width - Inches(0.05)

            _add_small_row_gauge(
                slide,
                pct_val,
                gauge_left,
                gauge_top,
                gauge_width,
                gauge_height
            )

            cumulative_height += row_height

    # ---------- RESIZE CARD TO MATCH FINAL TABLE HEIGHT ----------
    card.width = table_width + Inches(1.6)
    card.height = total_table_height + Inches(0.70)
    card.left = table_left - Inches(0.40)
    card.top = table_top - Inches(0.40)

    # ---------- FOOTER ----------
    _add_footer(prs, slide, slide_number)

    return slide


# build ppts

In [123]:
from datetime import datetime

def format_slide_title(base_title, overall_end_date, period_start, period_end):
    overall_dt = datetime.strptime(overall_end_date, "%Y-%m-%d")
    overall_month_year = overall_dt.strftime("%B %Y")

    period_dt = datetime.strptime(period_end, "%Y-%m-%d")
    period_month_year = period_dt.strftime("%B %Y")

    mapping = {
        # ✅ OVERALL SLIDES (unchanged)
        "Overall – State":          f"Statewise Testing (Till {overall_month_year})",
        "Overall – Specialization": f"Speciality wise Testing (Till {overall_month_year})",
        "Overall – Age Groups":     f"Age Group Testing (Till {overall_month_year})",
        "Overall – Gender":         f"Genderwise Testing (Till {overall_month_year})",

        # ✅ PERIOD SLIDES (NOW CONSISTENT)
        "Period – State":           f"Statewise Testing ({period_month_year})",
        "Period – Specialization":  f"Speciality wise Testing ({period_month_year})",
        "Period – Age Groups":      f"Age Group Testing ({period_month_year})",
        "Period – Gender":          f"Genderwise Testing ({period_month_year})",
    }

    return mapping.get(base_title, base_title)

def build_ppt_from_tables(
    tables_dict,
    output_file="middle_slides.pptx",
    pharma_filter=None
):
    prs = Presentation()
    blank = prs.slide_layouts[6]

    overall_df = tables_dict["RAW_OVERALL"]
    period_df  = tables_dict["RAW_PERIOD"]
    period_start = tables_dict["PERIOD_START"]
    period_end   = tables_dict["PERIOD_END"]

    # 0️⃣ Overview slide
    overview_data = compute_overview_metrics(overall_df)  # your existing function
    add_overview_slide(prs, overview_data)

    # 1️⃣ Overall slides
    overall_end_date = tables_dict["PERIOD_END"]
    for title, df in tables_dict.items():
        if title.startswith("Overall"):
            new_title = format_slide_title(title, overall_end_date, period_start, period_end)
            df_to_slide(prs, df, new_title)

    # 2️⃣ Period header slide - modern section divider
    slide = prs.slides.add_slide(blank)
    slide_number = len(prs.slides)
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    section_title_box = slide.shapes.add_textbox(
        Inches(0.5), slide_height / 2 - Inches(0.8),
        slide_width - Inches(1), Inches(1.2)
    )
    tf_sec = section_title_box.text_frame
    p_sec = tf_sec.paragraphs[0]
    # Derive month name from period_end
    period_dt = datetime.strptime(period_end, "%Y-%m-%d")
    period_month = period_dt.strftime("%B %Y")  # e.g. "January 2025"
    p_sec.text = f"{period_month} Insights"
    p_sec.font.size = Pt(40)
    p_sec.font.bold = True
    p_sec.font.color.rgb = TEXT_DARK
    p_sec.alignment = PP_ALIGN.CENTER

    sub_box = slide.shapes.add_textbox(
        Inches(0.5), slide_height / 2 + Inches(0.1),
        slide_width - Inches(1), Inches(0.6)
    )
    tf_sub = sub_box.text_frame
    p_sub = tf_sub.paragraphs[0]
    p_sub.text = f"{period_start} to {period_end}"
    p_sub.font.size = Pt(20)
    p_sub.font.color.rgb = TEXT_MUTED
    p_sub.alignment = PP_ALIGN.CENTER

    accent = slide.shapes.add_shape(
        MSO_AUTO_SHAPE_TYPE.RECTANGLE,
        Inches(3), slide_height / 2 + Inches(0.9),
        slide_width - Inches(6), Inches(0.08)
    )
    accent.fill.solid()
    accent.fill.fore_color.rgb = TEAL_PRIMARY
    accent.line.fill.background()

    _add_footer(prs, slide, slide_number)

    # 3️⃣ Period slides
    for title, df in tables_dict.items():
        if title.startswith("Period"):
            new_title = format_slide_title(title, overall_end_date, period_start, period_end)
            df_to_slide(prs, df, new_title)

    prs.save(output_file)
    return output_file

# pipeline

In [124]:

def merge_ppts(output_file, *ppt_files):
    powerpoint = win32.Dispatch("PowerPoint.Application")
    powerpoint.Visible = True

    base = powerpoint.Presentations.Open(os.path.abspath(ppt_files[0]))

    for ppt in ppt_files[1:]:
        pres = powerpoint.Presentations.Open(os.path.abspath(ppt))
        pres.Slides.Range().Copy()
        base.Slides.Paste()
        pres.Close()

    base.SaveAs(os.path.abspath(output_file))
    base.Close()
    powerpoint.Quit()

    print("✅ Final PPT created:", output_file)


In [125]:
from datetime import datetime, timedelta
import calendar

def get_last_month_range():
    """Returns (start_date, end_date) for the previous month."""
    today = datetime.today()
    first_this_month = today.replace(day=1)
    last_prev_month = first_this_month - timedelta(days=1)
    start_prev_month = last_prev_month.replace(day=1)
    return start_prev_month.strftime("%Y-%m-%d"), last_prev_month.strftime("%Y-%m-%d")

def build_final_tables(conn,
                       pharma_filter="ALL",
                       overall_threshold=40,
                       period_threshold=20):

    # Overall data = FULL till current month
    overall_df = load_raw_data(conn, load_all=True, pharma_filter=pharma_filter)
    overall_df["category"] = overall_df["category"].astype(str)

    # Period data = LAST MONTH ONLY
    start_date, end_date = get_last_month_range()
    period_df = load_raw_data(conn, start_date, end_date, pharma_filter, load_all=False)
    period_df["category"] = period_df["category"].astype(str)

    return {
        # OVERALL tables
        "Overall – State":          build_group_table(overall_df, "State"),
        "Overall – Specialization": build_group_table(overall_df, "specialization", overall_threshold),
        "Overall – Age Groups":     build_group_table(overall_df, "age_group"),
        "Overall – Gender":         build_group_table(overall_df, "gender"),

        # PERIOD tables
        "Period – State":           build_group_table(period_df, "State"),
        "Period – Specialization":  build_group_table(period_df, "specialization", period_threshold),
        "Period – Age Groups":      build_group_table(period_df, "age_group"),
        "Period – Gender":          build_group_table(period_df, "gender"),

        # Raw refs
        "RAW_OVERALL": overall_df,
        "RAW_PERIOD": period_df,
        "PERIOD_START": start_date,
        "PERIOD_END": end_date,
        "PHARMA_FILTER": pharma_filter
    }


In [126]:
if __name__ == "__main__":

    OVERALL_THRESHOLD = 1200   # specializations below this → Others (overall)
    PERIOD_THRESHOLD  = 50   # specializations below this → Others (period)

    tables = build_final_tables(
        conn,
        pharma_filter="bayer pharma",
        overall_threshold=OVERALL_THRESHOLD,
        period_threshold=PERIOD_THRESHOLD
    )

    build_ppt_from_tables(
        tables,
        output_file="middle_slides.pptx",
        pharma_filter="bayer pharma"
    )

    merge_ppts(
        "final_output.pptx",
        "template1.pptx",
        "middle_slides.pptx",
        "template2.pptx"
    )


✅ Final PPT created: final_output.pptx
